# Experiments

### Paths

In [1]:
import sys

root_path = ".."  # top-level of the repository

sys.path.append(root_path)

### Setup

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

import os
from pathlib import Path
from tempfile import mkdtemp

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from pymks import TwoPointCorrelation
from skimage import io
from skimage.color import rgb2gray
from skimage.feature import canny
from skimage.filters import threshold_otsu
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.compose import TransformedTargetRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn import metrics
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PowerTransformer, FunctionTransformer, QuantileTransformer

from src.metrics import regression_report, regression_score
from src.models.baselines import MeanBaseline, MedianBaseline
from src.fractal import compute_fractal_dimension
from src.thresholding import count_pixel_values, otsu_threshold, local_threshold_otsu
from src.utils import load_image_batch, sample_one, to_batch_function

In [4]:
sns.set()
plt.rcParams["figure.figsize"] = (16, 10)

In [5]:
cachedir = mkdtemp()  # for sklearn pipelines

# Dataset

In [6]:
mesurements_df = pd.read_csv("../data/dataset.csv")
mesurements_df.head(3)

,Sample,Hardness,File,Location,Code
0,1-101-2,129.9,../data/images/processed/1-101-2.jpg,top-right,1-101
1,1-101-4,130.0,../data/images/processed/1-101-4.jpg,bottom-left,1-101
2,1-102-2,137.6,../data/images/processed/1-102-2.jpg,top-right,1-102


Split dataset into train and test using sample code, to prevent information leakage.

In [7]:
train_codes, test_codes = train_test_split(mesurements_df["Code"].unique(), test_size=0.3, random_state=7)
len(train_codes), len(test_codes)

(176, 76)

In [8]:
train_df = mesurements_df.loc[mesurements_df["Code"].isin(train_codes)]
test_df = mesurements_df.loc[mesurements_df["Code"].isin(test_codes)]

train_images = load_image_batch(train_df["File"].tolist())
test_images = load_image_batch(test_df["File"].tolist())

# Baselines

Baseline results to compare to. <br>
Those just return mean and median values of the training data <br>
Only `MeanBaseline` is included in the paper.

### Mean Baseline

In [9]:
mean_baseline = MeanBaseline()
mean_baseline.fit(x=None, y=train_df["Hardness"].values)

In [10]:
predictions = mean_baseline.predict(test_df["Hardness"])  # input variable matters just to keep consistent shape
print(regression_report(y_true=test_df["Hardness"].values, y_pred=predictions))

                                Absolute                        Normalized                      

Mean Squared Error:             66.6716                         8.2406                          
Root Mean Squared Error:        8.1653                          1.0092                          
Mean Absolute Error:            6.4395                          0.7959                          
Median Absolute Error:          5.3586                          0.6623                          
Max Error:                      24.2586                         2.9983                          
R2                                                              -0.0185                         


                                True                            Predicted                       

Mean:                           136.9574                        138.0586                        
std:                            8.0907                          0.0000                          



# Otsu-Based Index

Experiment using otsu-based index as described in the paper. <br>
Targets are scaled using `StandardScaler` and computation is done in sklearn `Pipeline` object.

In [11]:
target_pipeline = Pipeline([
    ("scalar", StandardScaler()),
])

pipeline = Pipeline([
    ("color_to_grey", FunctionTransformer(rgb2gray)),
    ("otsu", FunctionTransformer(to_batch_function(otsu_threshold))),
    ("count_pixels", FunctionTransformer(count_pixel_values)),
    ("model", TransformedTargetRegressor(LinearRegression(), transformer=target_pipeline))
])

In [12]:
%%time

pipeline = pipeline.fit(train_images, train_df["Hardness"].values)

CPU times: user 7.02 s, sys: 10.1 s, total: 17.1 s
Wall time: 17.1 s


In [13]:
predictions = pipeline.predict(test_images)

In [14]:
print(regression_report(y_true=test_df["Hardness"].values, y_pred=predictions.flatten()))

                                Absolute                        Normalized                      

Mean Squared Error:             52.2930                         6.4634                          
Root Mean Squared Error:        7.2314                          0.8938                          
Mean Absolute Error:            5.6436                          0.6975                          
Median Absolute Error:          4.4881                          0.5547                          
Max Error:                      23.2307                         2.8713                          
R2                                                              0.2011                          


                                True                            Predicted                       

Mean:                           136.9574                        137.4273                        
std:                            8.0907                          3.1992                          



# Fractal dimension index

*Note*: This is not reproduction of the results in the paper (it can be found below). <br>
The parameter `sigma` of Canny detector was set to 1, and not 0.8. <br>

Exact reproduction is provised further in this notebook.

In [15]:
def canny_filter(x):
    """Shorthand to use canny with non-default sigma value and to_batch_function wrapper"""
    return canny(x, sigma=0.8)

target_pipeline = Pipeline([
    ("scalar", StandardScaler()),
])

pipeline = Pipeline([
    ("color_to_grey", FunctionTransformer(rgb2gray)),
    ("canny", FunctionTransformer(to_batch_function(canny_filter))),
    ("fractal_dimension", FunctionTransformer(to_batch_function(compute_fractal_dimension))),
    ("model", TransformedTargetRegressor(LinearRegression(), transformer=target_pipeline))
])

In [16]:
%%time

pipeline = pipeline.fit(train_images, train_df["Hardness"].values)

CPU times: user 1min 3s, sys: 6.88 s, total: 1min 10s
Wall time: 1min 10s


In [17]:
predictions = pipeline.predict(test_images)

In [18]:
print(regression_report(y_true=test_df["Hardness"].values, y_pred=predictions.flatten()))

                                Absolute                        Normalized                      

Mean Squared Error:             45.1667                         5.5826                          
Root Mean Squared Error:        6.7206                          0.8307                          
Mean Absolute Error:            5.1638                          0.6382                          
Median Absolute Error:          4.1153                          0.5086                          
Max Error:                      27.7682                         3.4321                          
R2                                                              0.3100                          


                                True                            Predicted                       

Mean:                           136.9574                        137.4277                        
std:                            8.0907                          3.8298                          



# 2-Point Correlation

In [19]:
target_pipeline = Pipeline([
    ("scalar", StandardScaler()),
])

pipeline = Pipeline([
    ("color_to_grey", FunctionTransformer(rgb2gray)),
    ("ostu_threshold", FunctionTransformer(to_batch_function(otsu_threshold))),
    ("2p_correlation", TwoPointCorrelation(periodic_boundary=True, cutoff=600)),
    ("flatten", FunctionTransformer(lambda x: x.reshape(x.shape[0], x.shape[1] * x.shape[2]))),  # flatten image, not batch of images
    ("pca", PCA(n_components=50)),
    ("model", TransformedTargetRegressor(LinearRegression(), transformer=target_pipeline))
])

In [20]:
%%time

pipeline = pipeline.fit(train_images, train_df["Hardness"].values)
predictions = pipeline.predict(test_images)

CPU times: user 4min 33s, sys: 1min 3s, total: 5min 37s
Wall time: 1min 21s


In [21]:
# results might vary from run to run, because of random initialization, but should be on avarage the same as reported
print(regression_report(y_true=test_df["Hardness"].values, y_pred=predictions.flatten()))

                                Absolute                        Normalized                      

Mean Squared Error:             55.9808                         6.9192                          
Root Mean Squared Error:        7.4820                          0.9248                          
Mean Absolute Error:            5.8748                          0.7261                          
Median Absolute Error:          4.8804                          0.6032                          
Max Error:                      24.8302                         3.0690                          
R2                                                              0.1448                          


                                True                            Predicted                       

Mean:                           136.9574                        137.6545                        
std:                            8.0907                          3.2818                          

